In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

C:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\harsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
# Step 1: Load the pre-trained model
model = load_model("/content/drive/MyDrive/ProfessorProject/2ChartClassification/Chart.h5")

In [ ]:
# Step 2: Define paths to the evaluation data folders
evaluation_FigQA_folder = ""

In [ ]:
# Total images in FigQA folder
classes = os.listdir(evaluation_FigQA_folder)
print("Total number of images per class: ")
for class_name in classes:
    class_path = os.path.join(evaluation_FigQA_folder, class_name)
    num_images = len(os.listdir(class_path))
    print(f"Class: {class_name}, Number of images: {num_images}")

Total number of images per class: 
Class: bar, Number of images: 15581
Class: line, Number of images: 2195
Class: pie, Number of images: 541


In [ ]:
# Step 3: Prepare the evaluation data
img_width, img_height = 224, 224  # Adjust the image size based on your model requirements
batch_size = 32

# Use ImageDataGenerator to load and preprocess the evaluation images
datagen = ImageDataGenerator(rescale=1.0 / 255.0)



if len(evaluation_FigQA_folder) != 0:
  evaluation_raw_generator = datagen.flow_from_directory(
      evaluation_FigQA_folder,
      target_size=(img_width, img_height),
      batch_size=batch_size,
      class_mode='categorical',
      shuffle=False
  )


Found 18317 images belonging to 3 classes.


In [ ]:
# Step 4: Evaluate the model
# 4.1 Evaluation on FigQA dataset
if len(evaluation_FigQA_folder) != 0:
  num_samples = evaluation_raw_generator.samples
  steps_per_epoch = int(np.ceil(num_samples / batch_size))

  # Get the class labels in the correct order (important for per class accuracy)
  class_labels = list(evaluation_raw_generator.class_indices.keys())

  # Evaluate the model and make predictions
  predictions = model.predict_generator(evaluation_raw_generator, steps=steps_per_epoch)
  predicted_labels = np.argmax(predictions, axis=1)

  # Get the ground-truth labels
  ground_truth_labels = evaluation_raw_generator.classes

  # Calculate per-class accuracy
  per_class_accuracy_Raw = {}
  for i, class_label in enumerate(class_labels):
      class_indices = np.where(ground_truth_labels == i)[0]
      correct_predictions = np.sum(predicted_labels[class_indices] == i)
      total_samples = len(class_indices)
      per_class_accuracy_Raw[class_label] = correct_predictions / total_samples

C:\Users\harsh\AppData\Local\Temp\ipykernel_884\3911544973.py:11: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(evaluation_raw_generator, steps=steps_per_epoch)


14428 15581 bar
2192 2195 line
541 541 pie


In [ ]:
# Step 5: Extract evaluation metrics
# 5.1: On FigQA dataset
if len(evaluation_FigQA_folder) != 0:
  evaluation_results = model.evaluate_generator(evaluation_raw_generator, steps=steps_per_epoch)
  loss_rawData = evaluation_results[0]
  accuracy_rawData = evaluation_results[1]

C:\Users\harsh\AppData\Local\Temp\ipykernel_884\395925203.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluation_results = model.evaluate_generator(evaluation_raw_generator, steps=steps_per_epoch)


In [ ]:

# Step 6: Print the evaluation summary
# 6.1: On FigQA dataset
if len(evaluation_FigQA_folder) != 0:
  print("Evaluation Summary on FigQA dataset:")
  print("Overall Loss:", loss_rawData)
  print("Overall Accuracy:", accuracy_rawData)
  print("Per Class Accuracy:")
  for class_name, acc in per_class_accuracy_Raw.items():
      print(f"{class_name}: {acc}")
  print('')
  print('')

Evaluation Summary on ChartQA dataset:
Overall Loss: 0.14707538485527039
Overall Accuracy: 0.9368892312049866
Per Class Accuracy:
bar: 0.9259996149156023
line: 0.9986332574031891
pie: 1.0


